### Artificial Neural Network

In [1]:
# create a perceptron network
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

In [2]:
# load data
iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target == 0) # Iris setosa

In [3]:
# Train model
per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

Perceptron(random_state=42)

In [6]:
# predict new values
X_new = [[2, 0.5], [3, 1]]
y_pred = per_clf.predict(X_new) # predicts True and False for these 2 flowers
y_pred

array([ True, False])

In [20]:
# create a nn with 3 hidden layers with 50 neurons each
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], random_state=42)
pipeline = make_pipeline(StandardScaler(), mlp_reg)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False) # about 0.505

In [ ]:
# using keras load fashion mnist data
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [ ]:
# scale pixel intensities
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# build model 2 hidden layers and with 10 output classes
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [ ]:
# display the summary
model.summary()

In [ ]:
model.layers
model.layers[1].name == 'dense'

In [ ]:
weights, biases = model.layers[1].get_weights()
biases

In [ ]:
# compile the model with loss function and optimizer
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
# train the model
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

In [ ]:
# show learning curves of loss, optimizers and metrics
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 29], ylim=[0, 1], grid=True, xlabel="Epoch",
    style=["r--", "g--.", "b-", "b-*"])
plt.show()

In [ ]:
# evaluate how well model does on test set
model.evaluate(X_test, y_test)

In [ ]:
# Make predictions
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
# classes with highest probability
import numpy as np

y_pred = y_proba.argmax(axis=-1)
y_pred

In [ ]:
np.array(class_names)[y_pred]

In [ ]:
y_new = y_test[:3]
y_new

### Build regression MLP NN

In [ ]:
# build NN for regression
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test, rmse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

### Functional Model Wide and Deep NN

In [26]:
normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

In [27]:
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)

model = tf.keras.Model(inputs=[input_], outputs=[output])

In [29]:
# 2 inputs 2 different paths
input_wide = tf.keras.layers.Input(shape=[5]) # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[6]) ## features 2 to 7
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

In [32]:
# compile build train model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20,
                    validation_data=((X_valid_wide, X_valid_deep), y_valid))
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)
y_pred = model.predict((X_new_wide, X_new_deep))

Epoch 1/20
363/363 [==============================] - 0s 810us/step - loss: 0.4218 - root_mean_squared_error: 0.6495 - val_loss: 0.4043 - val_root_mean_squared_error: 0.6358
Epoch 2/20
363/363 [==============================] - 0s 537us/step - loss: 0.3893 - root_mean_squared_error: 0.6239 - val_loss: 1.1397 - val_root_mean_squared_error: 1.0676
Epoch 3/20
363/363 [==============================] - 0s 726us/step - loss: 0.3859 - root_mean_squared_error: 0.6212 - val_loss: 6.3081 - val_root_mean_squared_error: 2.5116
Epoch 4/20
363/363 [==============================] - 0s 508us/step - loss: 0.3940 - root_mean_squared_error: 0.6277 - val_loss: 7.3957 - val_root_mean_squared_error: 2.7195
Epoch 5/20
363/363 [==============================] - 0s 494us/step - loss: 0.4124 - root_mean_squared_error: 0.6422 - val_loss: 2.2125 - val_root_mean_squared_error: 1.4874
Epoch 6/20
363/363 [==============================] - 0s 509us/step - loss: 0.3600 - root_mean_squared_error: 0.6000 - val_loss: 1

In [33]:
# multiple outputs, aux output for regularization
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])

In [34]:
# each output needs own loss function
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

In [35]:
# train model
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

Epoch 1/20
363/363 [==============================] - 1s 883us/step - loss: 0.8469 - dense_6_loss: 0.7534 - dense_7_loss: 1.6886 - dense_6_root_mean_squared_error: 0.8680 - dense_7_root_mean_squared_error: 1.2995 - val_loss: 0.5361 - val_dense_6_loss: 0.4699 - val_dense_7_loss: 1.1319 - val_dense_6_root_mean_squared_error: 0.6855 - val_dense_7_root_mean_squared_error: 1.0639
Epoch 2/20
363/363 [==============================] - 0s 669us/step - loss: 0.3967 - dense_6_loss: 0.3802 - dense_7_loss: 0.5454 - dense_6_root_mean_squared_error: 0.6166 - dense_7_root_mean_squared_error: 0.7385 - val_loss: 0.5716 - val_dense_6_loss: 0.5660 - val_dense_7_loss: 0.6225 - val_dense_6_root_mean_squared_error: 0.7523 - val_dense_7_root_mean_squared_error: 0.7890
Epoch 3/20
363/363 [==============================] - 0s 640us/step - loss: 0.3695 - dense_6_loss: 0.3576 - dense_7_loss: 0.4765 - dense_6_root_mean_squared_error: 0.5980 - dense_7_root_mean_squared_error: 0.6903 - val_loss: 0.3419 - val_dense_

In [36]:
# evaluate the model
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

162/162 [==============================] - 0s 436us/step - loss: 0.3193 - dense_6_loss: 0.3134 - dense_7_loss: 0.3722 - dense_6_root_mean_squared_error: 0.5598 - dense_7_root_mean_squared_error: 0.6101


In [37]:
# predictions for each output
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

1/1 [==============================] - 0s 83ms/step


In [38]:
y_pred_main

array([[0.44477814],
       [1.2067366 ],
       [3.6383715 ]], dtype=float32)

###